In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import numpy as np
import pandas as  pd
import matplotlib.pyplot as plt
import sys, os

In [3]:
%load_ext qat.core.magic

In [4]:
#QPU connection
try:
    from qat.qlmaas import QLMaaSConnection
    connection = QLMaaSConnection()
    LinAlg = connection.get_qpu("qat.qpus:LinAlg")
    lineal_qpu = LinAlg()
except ImportError:
    from qat.qpus import PyLinalg
    lineal_qpu = PyLinalg()

Username: gferro
········


In [5]:
sys.path.append('/home/cesga/gferro/NEASQC/PhaseAmplification/')

In [6]:
def p(x):
    return x*x
def f(x):
    return np.sin(x)

In [7]:
from dataloading_module import LeftConditionalProbability, CRBS_gate
from qat.core.console import display
from qat.lang.AQASM import Program, RY, AbstractGate, QRoutine, X

In [8]:
from AuxiliarFunctions import TestBins, LeftConditionalProbability, get_histogram, PostProcessResults

In [27]:
#number of Qbits for the circuit
n_qbits = 6
#The number of bins 
m_bins = 2**n_qbits
LowerLimit = 0.0
UpperLimit = 1.0 

x_X, p_X = get_histogram(p, LowerLimit, UpperLimit, m_bins)
f_X = f(x_X)

In [ ]:
def LoadProbability_generator(NumbeOfQbits, *ProbabilityArray):

    qrout = QRoutine()
    qbits = qrout.new_wires(NumbeOfQbits)
    nbins = 2**NumbeOfQbits  
    
    for i in range(0, NumbeOfQbits):
        ConditionalProbability = LeftConditionalProbability(i, ProbabilityArray)
        Thetas = 2.0*(np.arccos(np.sqrt(ConditionalProbability)))
        
        if i == 0:
            qrout.apply(RY(Thetas[0]), qbits[0])
        else:
            for j, theta in enumerate(Thetas):
                bNumber = list(format(j, '0{}b'.format(i)))
                bList = [bool(int(i)) for i in bNumber]
                for m, k in enumerate(bList):
                    if k == False:
                        qrout.apply(X, qbits[m])
                c_i_RY = RY(theta).ctrl(i)
                qrout.apply(c_i_RY, qbits[:i], qbits[i])
                for m, k in enumerate(bList):
                    if k == False:
                        qrout.apply(X, qbits[m])
                #gate = CRBS_gate(i, j, theta)
                #qrout.apply(gate, qbits[:i+1]) 
    return qrout



In [ ]:
def LoadAG(NumbeOfQbits, ProbabilityArray):
    LoadP_Gate = AbstractGate(
        "P_Gate", 
        [int]+[float for i in ProbabilityArray],
        circuit_generator=LoadProbability_generator
    )
    return LoadP_Gate

In [ ]:

LoadP_Gate = AbstractGate('Llamo', [int]+[float for i in p_X])
LoadP_Gate.set_circuit_generator(LoadProbability_generator)

In [ ]:
p_X

In [ ]:
p_Gate= LoadP_Gate(n_qbits, *p_X)

In [28]:

qprog = Program()
qbits = qprog.qalloc(n_qbits)
for i in range(0, n_qbits):
    ConditionalProbability = LeftConditionalProbability(i, p_X)
    Thetas = 2.0*(np.arccos(np.sqrt(ConditionalProbability)))
    
    if i == 0:
        qprog.apply(RY(Thetas[0]), qbits[0])
    else:
        for j, theta in enumerate(Thetas):
            c_i_RY = RY(theta).ctrl(i)
            qprog.apply(c_i_RY, qbits[:i], qbits[i])
            #gate = CRBS_gate(i, j, theta)
            #qrout.apply(gate, qbits[:i+1]) 

In [29]:
circuit = qprog.to_circ()

In [30]:
%qatdisplay circuit --depth 0

In [17]:
for instruction in circuit:
    print(instruction)

Op(gate='_0', qbits=[0], type=0, cbits=None, formula=None, remap=None)
Op(gate='X', qbits=[0], type=0, cbits=None, formula=None, remap=None)
Op(gate='_2', qbits=[0, 1], type=0, cbits=None, formula=None, remap=None)
Op(gate='X', qbits=[0], type=0, cbits=None, formula=None, remap=None)
Op(gate='_4', qbits=[0, 1], type=0, cbits=None, formula=None, remap=None)
Op(gate='X', qbits=[0], type=0, cbits=None, formula=None, remap=None)
Op(gate='X', qbits=[1], type=0, cbits=None, formula=None, remap=None)
Op(gate='_6', qbits=[0, 1, 2], type=0, cbits=None, formula=None, remap=None)
Op(gate='X', qbits=[0], type=0, cbits=None, formula=None, remap=None)
Op(gate='X', qbits=[1], type=0, cbits=None, formula=None, remap=None)
Op(gate='X', qbits=[0], type=0, cbits=None, formula=None, remap=None)
Op(gate='_8', qbits=[0, 1, 2], type=0, cbits=None, formula=None, remap=None)
Op(gate='X', qbits=[0], type=0, cbits=None, formula=None, remap=None)
Op(gate='X', qbits=[1], type=0, cbits=None, formula=None, remap=Non

In [24]:
circuit.statistics()

{'nbqbits': 8,
 'size': 255,
 'gates': {'custom gate': 0,
  'RY': 1,
  'C-RY': 2,
  'C-C-RY': 4,
  'C-C-C-RY': 8,
  'C-C-C-C-RY': 16,
  'C-C-C-C-C-RY': 32,
  'C-C-C-C-C-C-RY': 64,
  'C-C-C-C-C-C-C-RY': 128},
 'measurements': 0,
 'resets': 0,
 'logic': 0,
 'breaks': 0,
 'remaps': 0,
 'gate_size': 255}

In [31]:
job = circuit.to_job()

In [32]:
result = lineal_qpu.submit(job)

Submitted a new batch: Job525


In [33]:
job = circuit.to_job()
result = lineal_qpu.submit(job)
P_results = PostProcessResults(result.join())

Submitted a new batch: Job526


In [34]:
P_results

,States,Probability,Amplitude
0,|000000>,0.124977,0.353521+0.000000j
1,|100000>,0.269070,-0.518719+0.000000j
2,|110000>,0.319112,-0.564900+0.000000j
3,|111000>,0.101668,-0.318854+0.000000j
4,|111100>,0.036103,-0.190009+0.000000j
5,|111110>,0.009265,-0.096253+0.000000j
6,|111111>,0.139806,-0.373906+0.000000j


In [35]:
np.isclose(P_results['Probability'], p_X).all()

ValueError: operands could not be broadcast together with shapes (7,) (64,) 